In [ ]:
import numpy as np
import pylab 
import copy
from qiskit.providers.basicaer import BasicAerr
from qiskit.aqua.algorithms import NumPyMinimumEigenSolver, VQE
from qiskit. aqua.components.optimizers import SLSQP
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.drivers import PySCFDriver
from qiskit.chemistry.core import Hamiltonian, QubitMappingType
from qiskit.chemistry.components.initial_states import HartreeFrok

In [ ]:
molecule = 'H .O .O -{O}; Li .O .O {O}'
distances = np.arange(0.5,4.25,0.25)
vqe_energies = []
hf_energies = []
exact_energies = []

In [ ]:
for i,d in enumerate(distances):
    print('step',i)

    #set up experiment 
    driverPySCFDriver(molecule.format(d/2), basis = 'sto3g')
    qmolecule = driver.run
    operator = Hamiltonian (qubit_mapping = QubitMappingType.PARITY, 
                            two_qubit_reduction = True, freeze_core= True, orbital_reduction = [-3,-2])
    qubit_op, aux_ops = operator.run(qmolecule)

    #exact classical result
    exact_result = NumPyMinimumEigenSolver(qubit_op, aux_operators=aux_ops).run()
    exact_result = operator.process_algorithm_result(exact_result)


    # VQE
    optimizer = SLSQP(maxiter = 100)
    initial_state = HartreeFock(operator.molecule_info['num_orbitals'], 
                                operator.molecule_info['num_particles'],
                                qubit_mapping = operator.qubit_mapping, 
                                two_qubit_reduction = operator ._two_qubit_reduction)
    var_form = UCCSD(num_orbitals = operator.molecule_info['num_orbitals'], 
                     num_particles = operator.molecule_info['num_particles'],
                     initial_state = initial_state, 
                     qubit_mapping = operator._qubit_mapping, 
                     two_qubit_reduction = operator._two_qubit_reduction)
    algo = VQE (qubit_op, var_form, optimizer, aux_operators = aux_ops)
    vqe_result = algo.run(QuantumInstance(BasicAer.get_backend('statevector_simulaotr')))
    vqe_result = operator.process_algorithm_result(vqe_result)

    exact_energies =.append(exact_result.energy)
    vqe_energies.append(vqe_result.energy)
    hf_energies.append(vqe_result.hartree_fock_energy)
